# 5. Делаем вместе

# Work4WalmarT



# https://www.kaggle.com/c/walmart-recruiting-trip-type-classification/data

In [174]:
import math
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt

import catboost
from catboost import Pool, CatBoostClassifier

from sklearn.model_selection import train_test_split

from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.datasets import make_blobs

from sklearn.metrics import accuracy_score

warnings.filterwarnings("ignore")
%matplotlib inline


import logging
import yaml
from logging import config

import collections

In [30]:
# !pip install catboost

    100% |████████████████████████████████| 64.0MB 1.1MB/s eta 0:00:011
    100% |████████████████████████████████| 7.1MB 7.0MB/s eta 0:00:011
  Stored in directory: /home/care1e55/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [118]:
train = pd.read_csv('./walmart/train.csv')

TripType - a categorical id representing the type of shopping trip the customer made. This is the ground truth that you are predicting. TripType_999 is an "other" category.

VisitNumber - an id corresponding to a single trip by a single customer

Weekday - the weekday of the trip

Upc - the UPC number of the product purchased

ScanCount - the number of the given item that was purchased. A negative value indicates a product return.

DepartmentDescription - a high-level description of the item's department

FinelineNumber - a more refined category for each of the products, created by Walmart

In [119]:
train.head(5)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,6.811315e+10,-1,FINANCIAL SERVICES,1000.0
1,30,7,Friday,6.053882e+10,1,SHOES,8931.0
2,30,7,Friday,7.410811e+09,1,PERSONAL CARE,4504.0
3,26,8,Friday,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0
4,26,8,Friday,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0


In [120]:
train.shape

(647054, 7)

In [5]:
print(train.dtypes)
train.describe()

TripType                   int64
VisitNumber                int64
Weekday                   object
Upc                      float64
ScanCount                  int64
DepartmentDescription     object
FinelineNumber           float64
dtype: object


,TripType,VisitNumber,Upc,ScanCount,FinelineNumber
count,647054.000000,647054.000000,6.429250e+05,647054.000000,642925.000000
mean,58.584511,96167.640078,3.060698e+10,1.108878,3726.884567
std,157.635533,55545.485154,9.120134e+10,0.700776,2780.966158
min,3.000000,5.000000,8.340000e+02,-12.000000,0.000000
25%,27.000000,49268.000000,3.400001e+09,1.000000,1404.000000
50%,39.000000,97074.000000,7.050103e+09,1.000000,3352.000000
75%,40.000000,144316.000000,3.006531e+10,1.000000,5501.000000
max,999.000000,191347.000000,9.789707e+11,71.000000,9998.000000


In [121]:
for i in train.columns:
    print(i, train[i].isna().sum())

TripType 0
VisitNumber 0
Weekday 0
Upc 4129
ScanCount 0
DepartmentDescription 1361
FinelineNumber 4129


In [122]:
train.dropna(inplace=True)

In [123]:
for i in train.columns:
    print(i, train[i].isna().sum())

TripType 0
VisitNumber 0
Weekday 0
Upc 0
ScanCount 0
DepartmentDescription 0
FinelineNumber 0


In [124]:
print(train.dtypes)
train.describe()

TripType                   int64
VisitNumber                int64
Weekday                   object
Upc                      float64
ScanCount                  int64
DepartmentDescription     object
FinelineNumber           float64
dtype: object


,TripType,VisitNumber,Upc,ScanCount,FinelineNumber
count,642925.000000,642925.000000,6.429250e+05,642925.000000,642925.000000
mean,58.337228,96199.870353,3.060698e+10,1.110203,3726.884567
std,156.559892,55552.836758,9.120134e+10,0.701240,2780.966158
min,3.000000,5.000000,8.340000e+02,-12.000000,0.000000
25%,27.000000,49300.000000,3.400001e+09,1.000000,1404.000000
50%,39.000000,97116.000000,7.050103e+09,1.000000,3352.000000
75%,40.000000,144353.000000,3.006531e+10,1.000000,5501.000000
max,999.000000,191347.000000,9.789707e+11,71.000000,9998.000000


In [125]:
train = train.astype({'FinelineNumber' : 'int64',
             'VisitNumber' : 'int64',
             'TripType' : 'int64',
             'Upc' : 'int64',
             'ScanCount' : 'int64'}, inplace=True)


In [126]:
train.dtypes

TripType                  int64
VisitNumber               int64
Weekday                  object
Upc                       int64
ScanCount                 int64
DepartmentDescription    object
FinelineNumber            int64
dtype: object

In [127]:
train.head(5)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [132]:
mapping = train.groupby('VisitNumber')['ScanCount'].sum()
train['TotalScanCount'] = train['VisitNumber'].map(mapping)

In [133]:
x = train.groupby('VisitNumber')['DepartmentDescription'].apply(lambda grp: grp.value_counts().nlargest(1)).reset_index()
mapping = x.groupby('VisitNumber')['level_1'].unique()
train['Top1Department'] = train['VisitNumber'].map(mapping)

In [134]:
x = train.groupby('VisitNumber')['FinelineNumber'].apply(lambda grp: grp.value_counts().nlargest(1)).reset_index()
mapping = x.groupby('VisitNumber')['level_1'].unique()
train['Top1FinelineNumber'] = train['VisitNumber'].map(mapping)

In [135]:
mapping = train.groupby('VisitNumber')['FinelineNumber'].nunique()
train['#UniqueSubCategories'] =  train['VisitNumber'].map(mapping)

In [142]:
train.head(10)

,TripType,Weekday,TotalScanCount,Top1Department,Top1FinelineNumber,#UniqueSubCategories
0,999,Friday,-1,[FINANCIAL SERVICES],[1000],1
1,30,Friday,2,[SHOES],[8931],2
2,30,Friday,2,[SHOES],[8931],2
3,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16
4,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16
5,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16
6,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16
7,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16
8,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16
9,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16


In [141]:
train1 = train.drop(['VisitNumber', 'Upc', 'DepartmentDescription' , 'FinelineNumber', 'ScanCount'], axis=1)


In [145]:
train.head(10)

,TripType,Weekday,TotalScanCount,Top1Department,Top1FinelineNumber,#UniqueSubCategories
0,999,Friday,-1,[FINANCIAL SERVICES],[1000],1
1,30,Friday,2,[SHOES],[8931],2
2,30,Friday,2,[SHOES],[8931],2
3,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16
4,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16
5,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16
6,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16
7,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16
8,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16
9,26,Friday,27,[PAINT AND ACCESSORIES],[1017],16


In [147]:
train['Top1FinelineNumber'] = train['Top1FinelineNumber'].apply(str)
train['Top1Department'] = train['Top1Department'].apply(str)

In [148]:
train.drop_duplicates(keep=False, inplace=True)

In [157]:
train['Top1Department'] = train['Top1Department'].apply(lambda x: x.replace("[\'",""))
train['Top1Department'] = train['Top1Department'].apply(lambda x: x.replace("\']",""))
train['Top1FinelineNumber'] = train['Top1FinelineNumber'].apply(lambda x: x.replace("[",""))
train['Top1FinelineNumber'] = train['Top1FinelineNumber'].apply(lambda x: x.replace("]",""))

In [158]:
train.head(10)

,TripType,Weekday,TotalScanCount,Top1Department,Top1FinelineNumber,#UniqueSubCategories
116,9,Friday,1,MENS WEAR,8218,1
143,9,Friday,1,MENS WEAR,8225,1
164,9,Friday,1,IMPULSE MERCHANDISE,8296,1
202,7,Friday,1,FROZEN FOODS,4063,1
266,37,Friday,1,PRODUCE,3102,1
369,999,Friday,-1,HARDWARE,8410,1
375,9,Friday,2,HORTICULTURE AND ACCESS,6210,1
376,999,Friday,-1,HARDWARE,4513,1
437,9,Friday,1,INFANT CONSUMABLE HARDLINES,1009,1
467,8,Friday,2,DAIRY,1405,1


In [185]:
X_train, X_test, y_train, y_test = train_test_split(train, 
                                                   train['TripType'], test_size = 0.1, 
                                                   random_state=10, 
                                                   shuffle=True)

In [186]:
data_train = Pool(X_train.drop(['TripType'], axis=1), 
                  label=X_train.TripType,
                  cat_features=[0, 2, 3, 4])

data_eval = Pool(X_test.drop(['TripType'], axis=1), 
                 label=X_test.TripType,
                 cat_features=[0, 2, 3, 4])

In [200]:
ctb_params = {
    'depth': 6,
    'learning_rate': 0.1,
    'l2_leaf_reg': 3,
    'loss_function': 'MultiClass',
    'verbose': 1,
    'iterations': 500
}

model = CatBoostClassifier(**ctb_params, task_type="GPU", devices='0:1')

In [188]:
print(catboost.__version__)

0.21


In [201]:
model.fit(X = data_train, silent=False, eval_set=data_eval)

0:	learn: 2.2646013	test: 2.1239199	best: 2.1239199 (0)	total: 41.9ms	remaining: 20.9s
1:	learn: 2.0320197	test: 1.8758656	best: 1.8758656 (1)	total: 84.4ms	remaining: 21s
2:	learn: 1.8627509	test: 1.6997460	best: 1.6997460 (2)	total: 125ms	remaining: 20.8s
3:	learn: 1.7244326	test: 1.5668729	best: 1.5668729 (3)	total: 166ms	remaining: 20.6s
4:	learn: 1.6256744	test: 1.4633847	best: 1.4633847 (4)	total: 208ms	remaining: 20.6s
5:	learn: 1.5405688	test: 1.3753683	best: 1.3753683 (5)	total: 252ms	remaining: 20.8s
6:	learn: 1.4714472	test: 1.3015526	best: 1.3015526 (6)	total: 293ms	remaining: 20.7s
7:	learn: 1.4144679	test: 1.2432794	best: 1.2432794 (7)	total: 331ms	remaining: 20.3s
8:	learn: 1.3492372	test: 1.1783037	best: 1.1783037 (8)	total: 371ms	remaining: 20.2s
9:	learn: 1.3079174	test: 1.1365553	best: 1.1365553 (9)	total: 418ms	remaining: 20.5s
10:	learn: 1.2699517	test: 1.0987464	best: 1.0987464 (10)	total: 461ms	remaining: 20.5s
11:	learn: 1.2316941	test: 1.0616662	best: 1.0616662

96:	learn: 0.7072231	test: 0.6379817	best: 0.6379817 (96)	total: 4.25s	remaining: 17.7s
97:	learn: 0.7051867	test: 0.6372497	best: 0.6372497 (97)	total: 4.29s	remaining: 17.6s
98:	learn: 0.7032302	test: 0.6363229	best: 0.6363229 (98)	total: 4.34s	remaining: 17.6s
99:	learn: 0.7013376	test: 0.6359057	best: 0.6359057 (99)	total: 4.39s	remaining: 17.6s
100:	learn: 0.6973540	test: 0.6325186	best: 0.6325186 (100)	total: 4.42s	remaining: 17.5s
101:	learn: 0.6958322	test: 0.6320070	best: 0.6320070 (101)	total: 4.48s	remaining: 17.5s
102:	learn: 0.6916028	test: 0.6288465	best: 0.6288465 (102)	total: 4.51s	remaining: 17.4s
103:	learn: 0.6894286	test: 0.6270095	best: 0.6270095 (103)	total: 4.55s	remaining: 17.3s
104:	learn: 0.6872476	test: 0.6255612	best: 0.6255612 (104)	total: 4.61s	remaining: 17.3s
105:	learn: 0.6860524	test: 0.6249386	best: 0.6249386 (105)	total: 4.66s	remaining: 17.3s
106:	learn: 0.6837439	test: 0.6234977	best: 0.6234977 (106)	total: 4.7s	remaining: 17.3s
107:	learn: 0.68289

192:	learn: 0.5952336	test: 0.5912221	best: 0.5912221 (192)	total: 8.56s	remaining: 13.6s
193:	learn: 0.5944278	test: 0.5908247	best: 0.5908247 (193)	total: 8.61s	remaining: 13.6s
194:	learn: 0.5932419	test: 0.5908771	best: 0.5908247 (193)	total: 8.67s	remaining: 13.6s
195:	learn: 0.5930652	test: 0.5909168	best: 0.5908247 (193)	total: 8.7s	remaining: 13.5s
196:	learn: 0.5922794	test: 0.5905133	best: 0.5905133 (196)	total: 8.74s	remaining: 13.4s
197:	learn: 0.5916420	test: 0.5904172	best: 0.5904172 (197)	total: 8.79s	remaining: 13.4s
198:	learn: 0.5912000	test: 0.5903769	best: 0.5903769 (198)	total: 8.83s	remaining: 13.4s
199:	learn: 0.5907739	test: 0.5904456	best: 0.5903769 (198)	total: 8.87s	remaining: 13.3s
200:	learn: 0.5900486	test: 0.5901671	best: 0.5901671 (200)	total: 8.91s	remaining: 13.3s
201:	learn: 0.5893353	test: 0.5901880	best: 0.5901671 (200)	total: 8.95s	remaining: 13.2s
202:	learn: 0.5888355	test: 0.5900532	best: 0.5900532 (202)	total: 8.99s	remaining: 13.2s
203:	learn:

285:	learn: 0.5308908	test: 0.5780609	best: 0.5780609 (285)	total: 12.9s	remaining: 9.63s
286:	learn: 0.5300724	test: 0.5778805	best: 0.5778805 (286)	total: 12.9s	remaining: 9.6s
287:	learn: 0.5284633	test: 0.5764478	best: 0.5764478 (287)	total: 13s	remaining: 9.55s
288:	learn: 0.5279403	test: 0.5759406	best: 0.5759406 (288)	total: 13s	remaining: 9.51s
289:	learn: 0.5276143	test: 0.5758384	best: 0.5758384 (289)	total: 13.1s	remaining: 9.46s
290:	learn: 0.5271646	test: 0.5757450	best: 0.5757450 (290)	total: 13.1s	remaining: 9.41s
291:	learn: 0.5266562	test: 0.5756866	best: 0.5756866 (291)	total: 13.1s	remaining: 9.37s
292:	learn: 0.5261080	test: 0.5755048	best: 0.5755048 (292)	total: 13.2s	remaining: 9.33s
293:	learn: 0.5256233	test: 0.5753145	best: 0.5753145 (293)	total: 13.3s	remaining: 9.29s
294:	learn: 0.5250809	test: 0.5754578	best: 0.5753145 (293)	total: 13.3s	remaining: 9.24s
295:	learn: 0.5241192	test: 0.5748936	best: 0.5748936 (295)	total: 13.3s	remaining: 9.19s
296:	learn: 0.5

381:	learn: 0.4735096	test: 0.5707578	best: 0.5703753 (350)	total: 17.5s	remaining: 5.42s
382:	learn: 0.4731222	test: 0.5707262	best: 0.5703753 (350)	total: 17.6s	remaining: 5.37s
383:	learn: 0.4726152	test: 0.5708805	best: 0.5703753 (350)	total: 17.6s	remaining: 5.33s
384:	learn: 0.4723257	test: 0.5710561	best: 0.5703753 (350)	total: 17.7s	remaining: 5.28s
385:	learn: 0.4719299	test: 0.5710367	best: 0.5703753 (350)	total: 17.7s	remaining: 5.24s
386:	learn: 0.4713626	test: 0.5711428	best: 0.5703753 (350)	total: 17.8s	remaining: 5.19s
387:	learn: 0.4706043	test: 0.5708414	best: 0.5703753 (350)	total: 17.8s	remaining: 5.15s
388:	learn: 0.4701235	test: 0.5709986	best: 0.5703753 (350)	total: 17.9s	remaining: 5.1s
389:	learn: 0.4698104	test: 0.5709773	best: 0.5703753 (350)	total: 17.9s	remaining: 5.06s
390:	learn: 0.4695567	test: 0.5710598	best: 0.5703753 (350)	total: 18s	remaining: 5.01s
391:	learn: 0.4690735	test: 0.5711469	best: 0.5703753 (350)	total: 18s	remaining: 4.96s
392:	learn: 0.4

474:	learn: 0.4302259	test: 0.5672312	best: 0.5664681 (471)	total: 22.1s	remaining: 1.17s
475:	learn: 0.4294866	test: 0.5672001	best: 0.5664681 (471)	total: 22.2s	remaining: 1.12s
476:	learn: 0.4291472	test: 0.5672234	best: 0.5664681 (471)	total: 22.3s	remaining: 1.07s
477:	learn: 0.4287462	test: 0.5672539	best: 0.5664681 (471)	total: 22.3s	remaining: 1.03s
478:	learn: 0.4286002	test: 0.5672552	best: 0.5664681 (471)	total: 22.4s	remaining: 980ms
479:	learn: 0.4282441	test: 0.5672643	best: 0.5664681 (471)	total: 22.4s	remaining: 933ms
480:	learn: 0.4278338	test: 0.5671215	best: 0.5664681 (471)	total: 22.4s	remaining: 887ms
481:	learn: 0.4271245	test: 0.5668024	best: 0.5664681 (471)	total: 22.5s	remaining: 840ms
482:	learn: 0.4265345	test: 0.5667108	best: 0.5664681 (471)	total: 22.5s	remaining: 793ms
483:	learn: 0.4261309	test: 0.5664841	best: 0.5664681 (471)	total: 22.6s	remaining: 747ms
484:	learn: 0.4257753	test: 0.5666452	best: 0.5664681 (471)	total: 22.6s	remaining: 700ms
485:	learn

In [197]:
model.classes_

array([  3,   4,   5,   6,   7,   8,   9,  12,  15,  18,  19,  20,  21,
        22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,
        35,  36,  37,  38,  39,  41,  42, 999])